# Import some stuff

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Retrieve the data

In [20]:
data = pd.read_csv('fifa21_train.csv')
data.shape

FileNotFoundError: [Errno 2] No such file or directory: 'fifa21_train.csv'

# Start the cleaning

## Quinn

In [21]:
Weight = [] 
for i in data['Weight']:
    Weight.append(i.replace('lbs' , ''))

data['Weight'] = Weight
data['Weight'] = data['Weight'].astype(np.number)
data['Weight'] = data['Weight']/2.2046

## Remove redundant columns

In [22]:
# Column 'Team & Contract' can go, because it only combines values for two other columns.
data = data.drop(columns=['Team & Contract'])

# Next, there are a bunch of columns whose values are factored into other columns (like 'Attacking', 'Movement' etc)
todrop = ['Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes']
data = data.drop(columns=todrop, axis = 1)



KeyError: "['Team & Contract'] not found in axis"

## Transform the 'Contract' column to only give the end date of the contract 

In [ ]:
contractend = []
for item in data['Contract']:
    item = item.replace(' On Loan', '')
    contractend.append(item[-4:])

data['Contract']=contractend



## Remove stars

In [ ]:
def removestar(dataframe, column):
    newvalues = []
    for item in dataframe[column]:
        item = item.replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    dataframe[column]=newvalues
    return dataframe


removestar(data,'IR')
removestar(data,'W/F')
removestar(data,'SM')

## Transform the 'loan date end' column to a loan (yes/no) column

In [ ]:
data['Loan Date End'] = data['Loan Date End'].fillna(0)

In [ ]:
data['Loan Date End'] = data['Loan Date End'].fillna(0)

loanyesno = []
for item in data['Loan Date End']:
    if item == 0:
        loanyesno.append(item)
    else: 
        loanyesno.append(1)

data['Loan Date End'] = loanyesno
data = data.rename(columns={'Loan Date End':'On Loan'})

data['On Loan']

## Remove the plusses and minuses on the last few rows

In [23]:
# Remove the plusses and minuses on the last few rows

toclean = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']

superlist = []

for column in toclean: 
    values = []
    for item in data[column]:
        if item[1] == '-' or item[1] == '+':
            values.append(int(item[0]))
        else: 
            values.append(int(item[:2]))
    superlist.append(values)


for i in range(len(toclean)):
    data[toclean[i]] = superlist[i]


TypeError: 'int' object is not subscriptable

## Convert height to cm

In [24]:
cmheight = []

for item in data['Height']:
    item = item.replace('\"', '')
    feetinches = item.split('\'')
    cmheight.append(round((int(feetinches[0]) * 30.48) + (int(feetinches[1]) * 2.54), 0))

AttributeError: 'float' object has no attribute 'replace'

In [25]:
data['Height'] = cmheight

ValueError: Length of values (0) does not match length of index (11701)

## Clean up currency cells

In [26]:
def cleancurrency(dataframe, column):
    newvalues = []
    for item in dataframe[column]:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues.append(int(item))
        else: 
            item = int(item)
            newvalues.append(int(item))
    dataframe[column]=newvalues
    return dataframe

cleancurrency(data, 'Value')
cleancurrency(data, 'Wage')
cleancurrency(data, 'Release Clause')


data['Value']

AttributeError: 'int' object has no attribute 'replace'

## Clean up Hits column (which contains some values with K in them)

In [27]:
newhits = []
for item in data['Hits']:
    if 'K' in item:
        item = item.replace('K','')
        item = float(item) * 1000
        newhits.append(int(item))
    else: 
        newhits.append(int(item))
data['Hits'] = newhits


TypeError: argument of type 'int' is not iterable

In [28]:
for item in data['Hits']:
    if not str(item).isnumeric(): 
        print(item)

In [29]:
joinednew = 0
for item in data['Joined']:
    item = int(item[-4:])
    print(item)

2015
2015
2019
2016
2018
2020
2019
2019
2018
2020
2019
2020
2020
2015
2020
2019
2019
2009
2018
2018
2019
2013
2018
2018
2020
2019
2015
2015
2019
2017
2020
2019


TypeError: 'float' object is not subscriptable

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11701 entries, 0 to 11700
Data columns (total 66 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              11701 non-null  int64  
 1   Name            11701 non-null  object 
 2   Age             11701 non-null  int64  
 3   Nationality     11701 non-null  object 
 4   Club            11683 non-null  object 
 5   BP              11701 non-null  object 
 6   Position        11413 non-null  object 
 7   Height          11701 non-null  float64
 8   Weight          11701 non-null  float64
 9   foot            11701 non-null  object 
 10  Growth          11701 non-null  int64  
 11  Joined          11660 non-null  object 
 12  On Loan         11701 non-null  int64  
 13  Value           11701 non-null  int64  
 14  Wage            11701 non-null  int64  
 15  Release Clause  11701 non-null  int64  
 16  Contract        11701 non-null  object 
 17  Attacking       11701 non-null 

In [32]:
# Transform the 'Contract' column to only give the end date of the contract 
contractend = []
for item in data['Contract']:
    if 'Free' in item:
        index = loc.item
    else:
        item = item.replace(' On Loan', '')
        item = item[-4:]
    contractend.append(int(item))

NameError: name 'loc' is not defined

In [35]:
i = 0
for item in data['Contract']:
    if item[-4:] == 'Free':
        i += 1
print(i)

244


In [37]:
pd.set_option('display.max_columns', None)

data

,ID,Name,Age,Nationality,Club,BP,Position,Height,Weight,foot,Growth,Joined,On Loan,Value,Wage,Release Clause,Contract,Attacking,Skill,Movement,Power,Mentality,Defending,Goalkeeping,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.0,73.029121,Right,1,"Jul 1, 2015",0,525000,4000,801000,2020,258,286,346,306,290,148,48,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,183.0,72.121927,Right,0,"Jan 16, 2015",1,8500000,23000,0,2020,365,375,404,372,313,77,55,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,163.0,60.782001,Right,0,"Jan 31, 2019",0,9000000,49000,15300000,2022,336,424,424,308,332,80,21,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,178.0,68.946748,Right,13,"Jul 1, 2016",0,275000,4000,694000,2021,242,259,282,277,257,168,42,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.0,68.039554,Right,8,"Jul 1, 2018",0,725000,2000,1400000,2021,249,280,324,280,294,185,52,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.0,76.204300,Left,5,"Feb 8, 2018",0,625000,3000,891000,2021,224,248,333,273,266,184,61,1589,347,3,2,Medium,Medium,1,75,28,56,59,60,69,3,48,48,48,55,52,52,52,55,54,54,54,58,56,56,56,58,63,60,60,60,63,63,61,61,61,63,16,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.0,78.925882,Right,4,"Jul 1, 2019",0,1600000,4000,2900000,2022,84,96,237,217,101,41,347,1123,387,2,1,Medium,Medium,1,72,67,66,72,40,70,3,28,28,28,24,26,26,26,24,24,24,24,25,25,25,25,25,25,26,26,26,25,24,26,26,26,24,69,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,168.0,64.864374,Right,8,"Jan 23, 2019",0,1100000,2000,1700000,2024,286,291,388,278,241,62,57,1603,337,3,3,High,Medium,1,76,60,60,72,26,43,6,62,62,62,67,65,65,65,67,65,65,65,66,58,58,58,66,49,44,44,44,49,45,35,35,35,45,17,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,188.0,81.193867,Right,7,NaN,0,0,0,0,2010,254,284,198,258,241,113,81,1429,400,3,1,NaN,NaN,3,72,63,70,72,57,66,3,68,68,68,71,68,68,68,71,72,72,72,71,71,71,71,71,66,68,68,68,66,64,60,60,60,64,25,68
